# Imports

In [1]:
import nengo
import numpy, math
import matplotlib.pyplot as plt

# Initializations

In [2]:
model = nengo.Network()

epsilon = 0.12
weights1 = numpy.random.uniform(-epsilon, epsilon, size = (2, 2))
weights2 = numpy.random.uniform(-epsilon, epsilon, size = (2, 2))


# Helper Functions

Sigmoid: Returns sigmoid value of the passed argument

In [3]:
def sigmoid(x):
    if x > -6:
        sig = 1/(1+math.exp(-x))
    else:
        sig = 0
    return sig


Sigmoid Gradient: Used in Weight Update during Back Propagation

In [4]:
def sigmoid_grad(x):
    sig_grad = sigmoid(x)*(1-sigmoid(x))
    return sig_grad


Delta. Difference between actual and expectation

In [5]:
def calculate_delta(actual, expectation):
    return numpy.subtract(actual, expectation)

Weight Update

In [6]:
def weight_update(weights1, weights2, delta):
    weights1 = numpy.subtract(weights1, delta)
    weights2 = numpy.subtract(weights2, delta)
    return weights1, weights2

# Model

In [7]:
def model_architecture(weights1, weights2, inp):
    with model:
        #input vector. Pixels to be identified
        inp =  [1, 0]
        Input = nengo.Node(inp)

        Input_Node_1 = nengo.Ensemble(100, dimensions = 1)
        Input_Node_2 = nengo.Ensemble(100, dimensions = 1)
        Inp_Hidd_Weights_1 = nengo.Ensemble(100, dimensions = 1)
        Inp_Hidd_Weights_2 = nengo.Ensemble(100, dimensions = 1)
        Hidden_Node_1 = nengo.Ensemble(100, dimensions = 1)
        Hidden_Node_2 = nengo.Ensemble(100, dimensions = 1)
        Hidd_Out_Weights_1 = nengo.Ensemble(100, dimensions = 1)
        Hidd_Out_Weights_2 = nengo.Ensemble(100, dimensions = 1)        
        Output_Node_1 = nengo.Ensemble(100, dimensions = 1)
        Output_Node_2 = nengo.Ensemble(100, dimensions = 1)

        nengo.Connection(Input[0], Input_Node_1)
        nengo.Connection(Input[1], Input_Node_2)
        
        
        #Multiply activation with weights. Inp_Hidden_Weights_x represents the weighted inputs of node x
        nengo.Connection(Input_Node_1, Inp_Hidd_Weights_1, transform = weights1[0][0])
        nengo.Connection(Input_Node_2, Inp_Hidd_Weights_1, transform = weights1[1][0])
        nengo.Connection(Input_Node_1, Inp_Hidd_Weights_2, transform = weights1[0][1])
        nengo.Connection(Input_Node_2, Inp_Hidd_Weights_2, transform = weights1[1][1])
        
        #Apply sigmoid in the hidden layer
        nengo.Connection(Inp_Hidden_Weights_1, Hidden_Node_1, function = sigmoid)
        nengo.Connection(Inp_Hidden_Weights_2, Hidden_Node_2, function = sigmoid)

        
        
        nengo.Connection(Hidden_Node_1, Hidd_Out_Weights_1, transform = weights2[0][0])
        nengo.Connection(Hidden_Node_2, Hidd_Out_Weights_1, transform = weights2[1][0])
        nengo.Connection(Hidden_Node_1, Hidd_Out_Weights_2, transform = weights2[0][1])
        nengo.Connection(Hidden_Node_2, Hidd_Out_Weights_2, transform = weights2[1][1])
        
        #Apply sigmoid in the hidden layer
        nengo.Connection(Hidd_Out_Weights_1, Output_Node_1, function = sigmoid)
        nengo.Connection(Hidd_Out_Weights_2, Output_Node_2, function = sigmoid)
        
        net2_1 = nengo.Probe(Inp_Hidd_Weights_1)
        net2_2 = nengo.Probe(Inp_Hidd_Weights_2)
        
        y2_1 = nengo.Probe(Hidden_Node_1)
        y2_2 = nengo.Probe(Hidden_Node_2)
        
        net3_1 = nengo.Probe(Hidd_Out_Weights_1)
        net3_2 = nengo.Probe(Hidd_Out_Weights_2)

        z3_1 = nengo.Probe(Output_Node_1, synapse = 0.01)
        z3_2 = nengo.Probe(Output_Node_2, synapse = 0.01)
    return model

# Running and Probing the Model
#### Runs the model for specified time. Returns average activity as output.

In [8]:
def run(model, time):
    sim = nengo.Simulator(model)
    sim.run(1.0)
    net2 = numpy.array([numpy.mean(sim.data[net2_1]), numpy.mean(sim.data[net2_2])])
    y2 = numpy.array([numpy.mean(sim.data[y2_1]), numpy.mean(sim.data[y2_2])])
    net3 = numpy.array([numpy.mean(sim.data[net3_1]), numpy.mean(sim.data[net3_2])])
    z3 = numpy.array([numpy.mean(sim.data[z3_1]), numpy.mean(sim.data[z3_2])])                        
    return net2, y2, net3, z3

#### Learning Weights

In [9]:
def back_prop(X, Y, num_iters, weights1, weights2):
    #print "Before Back-prop: ", weights2
    w2 = weights2
    D1 = numpy.zeros(numpy.shape(weights1))
    D2 = numpy.zeros(numpy.shape(weights2))
    for i in range(num_iters):
        for j in range(len(X)):
            (net2, y2, net3, z3) = feed_forward(X[j], Y[j], weights1, weights2)
            delta3 = (Y[j] - z3) * sigmoid_grad(net3)
            delWeights2 = numpy.outer(numpy.transpose(delta3), y2)
            delta2 = numpy.dot(delta3, weights2)*sigmoid_grad(net2)
            delWeights1 = numpy.outer(numpy.transpose(delta2), X[j])
            weights1 = weights1 + 0.01*delWeights1
            weights2 = weights2 + 0.01*delWeights2
        print "Iteration", i, "complete"
        #print w2 - weights2
    return (weights1, weights2)

In [54]:
model = model_architecture(weights1, weights2, [0, 1])
actual = run(model, 1.0)
expectation = [0, 1]
delta = calculate_delta(actual, expectation)
weights1, weights2 = weight_update(weights1, weights2, delta)

Building finished in 0:00:03.                                                   
Simulating finished in 0:00:14.                                                 
Building finished in 0:00:02.                                                   
Simulating finished in 0:00:13.                                                 
Building finished in 0:00:02.                                                   
Simulating finished in 0:00:18.                                                 
Building finished in 0:00:03.                                                   
Simulating finished in 0:00:22.                                                 
Building finished in 0:00:04.                                                   
Simulating finished in 0:00:18.                                                 
Building finished in 0:00:05.                                                   
Simulating finished in 0:00:25.                                                 
Building finished in 0:00:03

In [56]:
print actual

(-0.03563744682724565, 0.026418932761017436)
